In [9]:
import numpy as np
import cupy as cp
import math
import random
from matplotlib import pyplot as plt
from scipy.optimize import minimize
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'

## Basic function

In [10]:
class Bfun():
    '''creat a Bfun for every input factor, consisting of 16 b_i'''

    def __init__(self, phis, interval):
        self.phis = phis
        self.size = phis.shape[0]
        self.interval = interval

    def forward(self, s):
        b = np.zeros(self.size)
        for i in range(self.size):
            if self.phis[i] - self.interval < s < self.phis[i] + self.interval:
                b[i] = 1 / 2 * math.cos(s - self.phis[i]) + 1 / 2
            else:
                b[i] = 0
        return b


class PoissonLogPenaltyLoss():
    '''loss function with penalty term'''
    def __init__(self, c, P, m):
        self.c = c
        self.P = P
        self.m = m
        self.prediction = None
        self.targets = None

    def add_paras(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def loss(self, Ws):
        Ws = Ws.reshape((self.m, 16))
        _prediction = predict(self.inputs, Ws)
        loss = np.sum(-self.targets * _prediction + np.exp(_prediction))
        for i in range(self.m):
            loss += self.c * (Ws[i, :].reshape((1, 16)) @ self.P @ Ws[i, :].reshape((16, 1))
                              + math.sqrt(16) * np.sum(Ws[i, :] ** 2))
        return float(loss)



## Prepocessing Data

In [11]:
# load data
cell_num = 884
m = 3
trial_num = 77
positions = np.load('C:/ForLiYuan/Behaviour.npy', allow_pickle=True)
speed = np.load('C:/ForLiYuan/Speed.npy', allow_pickle=True)
dff_traces = np.load('C:/ForLiYuan/DFF_traces.npy', allow_pickle=True)
contexts = np.load('C:/ForLiYuan/Map_index.npy', allow_pickle=True)
pointer = np.load('C:/ForLiYuan/sync.npy', allow_pickle=True)

In [12]:
pointer

array([ 3205,  3583,  4132,  4778,  5141,  6201,  7032,  7656,  8299,
        8811,  9364,  9942, 10497, 11266, 12471, 13013, 13769, 14279,
       15046, 16737, 17504, 18045, 19141, 19846, 20466, 21060, 21597,
       22737, 23346, 24141, 24720, 25332, 25955, 26443, 27055, 27561,
       28129, 28729, 29258, 30168, 30778, 31356, 31937, 32597, 33239,
       33874, 34454, 35025, 35583, 36183, 36793, 37408, 37947, 38610,
       39334, 39950, 40572, 41130, 41726, 42242, 42818, 43282, 43829,
       44363, 44927, 45481, 46031, 46760, 47418, 47962, 49193, 50176,
       50812, 51473, 52083, 52792, 53386, 53993])

In [13]:
context = np.zeros(trial_num)
for i in range(4):
    context[contexts[i]] = i

In [14]:
context

array([0., 3., 1., 2., 0., 1., 0., 3., 2., 2., 3., 0., 3., 1., 0., 1., 2.,
       2., 0., 3., 0., 2., 0., 3., 2., 1., 1., 3., 0., 3., 2., 2., 1., 3.,
       0., 1., 2., 1., 2., 0., 1., 3., 2., 0., 1., 3., 3., 1., 2., 0., 3.,
       1., 0., 2., 3., 0., 1., 2., 3., 3., 1., 2., 0., 3., 0., 3., 1., 2.,
       1., 1., 0., 2., 2., 0., 3., 1., 0.])

In [15]:
# determine range of input factors
for i in range(trial_num):
    if i == 0:
        pos_start = np.min(positions[i])
        pos_end = np.max(positions[i])
        speed_min = np.min(speed[i])
        speed_max = np.max(speed[i])
    else:
        if np.min(positions[i]) < pos_start:
            pos_start = np.min(positions[i])
        if np.max(positions[i]) > pos_end:
            pos_end = np.max(positions[i])
        if np.min(speed[i]) < speed_min:
            speed_min = np.min(speed[i])
        if np.max(speed[i]) > speed_max:
            speed_max = np.max(speed[i])
[pos_start, pos_end], [speed_min, speed_max]

([400.0, 2000.0], [-0.11206820221684037, 128.51277310937513])

### rearrange data into desired format

In [19]:
%%time
lenth = []
for trial in range(trial_num):
    lenth.append(positions[trial].shape[0])
lenth = np.array(lenth)
total_lenth = np.sum(lenth)
inputs = np.zeros((total_lenth, 3))
outputs = dff_traces[:, pointer[0]:pointer[-1]]
for trial in range(trial_num):
    if trial == 0:
        inputs[0:lenth[0], 0] = positions[0]
        inputs[0:lenth[0], 1] = speed[0]
        inputs[0:lenth[0], 2] = context[0]
    elif trial == trial_num-1:
        a = np.sum(lenth[:-1])
        inputs[a:, 0] = positions[-1]
        inputs[a:, 1] = speed[-1]
        inputs[a:, 2] = context[-1]
    else:
        a = np.sum(lenth[:trial])
        b = np.sum(lenth[:trial+1])
        inputs[a:b, 0] = positions[trial]
        inputs[a:b, 1] = speed[trial]
        inputs[a:b, 2] = context[trial]
inputs = cp.array(inputs)

CPU times: total: 31.2 ms
Wall time: 8.88 ms


## GLM

### initializing hyperparameters

In [20]:
P = np.zeros((16, 16))
for i in range(16):
    P[i, i] = 2
for i in range(1, 16):
    P[i, i-1] = -1
for i in range(0, 15):
    P[i, i+1] = -1
c = 1
# input factors: positions, speed, context(0, 1, 2, 3)
phis = np.zeros((m, 16))
inter = np.zeros(m)
phis[0, :] = np.linspace(400, 2000, 16)[:]
phis[1, :] = np.linspace(-10, 130, 16)[:]
phis[2, :] = np.linspace(0, 3, 16)[:]
inter[0] = 1600 / 15 * 2
inter[1] = 140 / 15 * 2
inter[2] = 3 / 15 * 2
Bfuncs = []
for i in range(m):
    Bfuncs.append(Bfun(phis[i], inter[i]))
# parameters for Adam optimizer
lr = 0.001
beta1 = 0.8
beta2 = 0.9
# bins before and after used for determining one time point
d_before = 4
d_after = 3


### Minderer 2019

In [21]:
def predict(inputs, Ws):
    '''make predictions of dF/F based on fitted weights and inputs'''
    t = inputs.shape[0]
    m = inputs.shape[1]
    prediction = cp.zeros(t)
    for i in range(t):
        fs = 0
        for j in range(m):
            fs += cp.sum(Ws[j, :] * Bfuncs[j].forward(inputs[i, j]))
        prediction[i] = fs
    return prediction

def fit_one_neuron(inputs, outputs):
    m = inputs.shape[1]
    global c, P
#     optimizer = AdamOptimizer(lr, beta1, beta2)
    loss_fn = PoissonLogPenaltyLoss(c, P, m)
    Ws = cp.random.normal(loc=0, scale=0.1*math.sqrt(2/16), size=(m, 16))
    t = inputs.shape[0]
    loss_fn.add_paras(inputs, outputs)
    res = minimize(loss_fn.loss, Ws.flatten(), method='BFGS')
    Ws = res.x.reshape(m, 16)
    return Ws


In [22]:
fit_scale = int(0.7*total_lenth)
fit_inputs = inputs[0:fit_scale, :]
fit_outputs = outputs[:, 0:fit_scale]
test_inputs = inputs[fit_scale:, :]
test_outputs = outputs[:, fit_scale:]

In [24]:
# gaussian kernel
# import cv2
# kernel = cv2.getGaussianKernel(ksize=4, sigma=1)
i = cp.array([0, 1, 2, 3])
kernel = cp.exp(-(i-(4-1)/2)**2/2)
kernel = kernel / cp.sum(kernel*i)
fit_size = fit_inputs.shape[0]
m = fit_inputs.shape[1]
fit_inputs_smooth = cp.zeros(fit_inputs.shape)
for i in range(3):
    fit_inputs_smooth[i, :] = fit_inputs[i, :]
for i in range(3, fit_size):
    for j in range(m):
        fit_inputs_smooth[i, j] = cp.sum(kernel*fit_inputs[i-3:i+1, j])

In [25]:
test_size = test_inputs.shape[0]
test_inputs_smooth = cp.zeros(test_inputs.shape)
for i in range(3):
    test_inputs_smooth[i, :] = test_inputs[i, :]
for i in range(3, test_size):
    for j in range(m):
        test_inputs_smooth[i, j] = cp.sum(kernel*test_inputs[i-3:i+1, j])

### fit a GLM for each neuron

In [26]:
%%time
Ws_all = cp.zeros((cell_num, m, 16)) # weights for all neurons
Ws_con_all = cp.zeros((cell_num, m, 16))
shuff_times = 10
roll_len = cp.random.randint(5, 10, size=shuff_times)
print(roll_len)

for cell_id in range(1):
    Ws = fit_one_neuron(fit_inputs_smooth, fit_outputs[cell_id, :])
#     Ws_all[cell_id, :, :] = Ws[:, :]
#     # control fits for GLM
#     Ws_con = np.zeros((m, 16))
#     for turn in range(shuff_times):
#         Ws_con += fit_one_neuron(fit_inputs, np.roll(fit_outputs[cell_id, :], roll_len[turn]))
#     Ws_con /= shuff_times
#     Ws_con_all[cell_id, :, :] = Ws_con[:, :]
#     # plotting weights
#     plt.subplot(311)
#     plt.plot(Ws_con[0, :], label='fit_control')
#     plt.plot(Ws[0, :], label='fit')
#     plt.title('weights for position')
#     plt.legend()
#     plt.subplot(312)
#     plt.plot(Ws_con[1, :], label='fit_control')
#     plt.plot(Ws[1, :], label='fit')
#     plt.title('weights for speed')
#     plt.legend()
#     plt.subplot(313)
#     plt.plot(Ws_con[2, :], label='fit_control')
#     plt.plot(Ws[2, :], label='fit')
#     plt.title('weights for context')
#     plt.legend()
#     plt.suptitle('weights for neuron'+str(cell_id))
#     plt.savefig(str(cell_id)+'weights.jpg')
#     plt.show()

#     # plotting predictions
#     predict1 = np.exp(predict(test_inputs, Ws))
#     predict2 = np.exp(predict(test_inputs, Ws_con))
#     predictions[cell_id, :] = predict1[:]
#     predictions_con[cell_id, :] = predict2[:]
#     plt.plot(predict1, label='predict')
#     plt.plot(predict2, label='predict control')
#     plt.plot(test_outputs[cell_id, :], label='real')
#     plt.legend()
#     plt.xlabel('time(s)')
#     plt.ylabel('dff')
#     plt.show()

# plt.imshow(predictions.T)
# plt.colorbar()
# plt.title('predictions for trial'+str(trial_id))
# plt.show()

# plt.imshow(predictions_con.T)
# plt.colorbar()
# plt.title('predictions control for trial'+str(trial_id))
# plt.show()

# plt.imshow(predictions.T - predictions_con.T)
# plt.colorbar()
# plt.title('errors for trial'+str(trial_id))
# plt.show()

[8 6 9 9 8 5 7 8 7 7]


TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [ ]:
 # plotting predictions
cell_id = 1
turn = test_inputs_smooth.shape[0] // 50
bins = 50
for i in range(turn):
    predict1 = np.exp(predict(test_inputs_smooth[i*bins:(i+1)*bins], Ws))
#     predict2 = np.exp(predict(test_inputs[i*bins:(i+1)*bins], Ws_con))
#     predictions[cell_id, :] = predict1[:]
#     predictions_con[cell_id, :] = predict2[:]
    plt.plot(predict1, label='predict')
#     plt.plot(predict2, label='predict control')
    plt.plot(test_outputs[cell_id, i*bins:(i+1)*bins], label='real')
    plt.legend()
    plt.xlabel('time(s)')
    plt.ylabel('dff')
    plt.show()